In [1]:
import laserembeddings as le
import xml.etree.ElementTree as ElementTree
from gensim.models.keyedvectors import KeyedVectors
import os
import pickle
import heapq
import numpy as np
from collections import defaultdict

In [28]:
#Stop-words initialization
en_stop,hn_stop,bn_stop = [],[],[]
f = open('eng_stop_words')
for l in f:
    en_stop.append(l.strip())
f = open('latest_hn_stop_words')
for l in f:
    hn_stop.append(l.strip())
f = open('latest_bn_stop_words')
for line in f:
    bn_stop.append(line.strip())

In [4]:
###English corpus generating as a list of lists(sentence)
tot_en=0
final_list_eng = []
for ids in range(176,226):
    for root,dirs,files in os.walk('thesis/rel/'+str(ids)+'/en/'):
        for file in files:
            file_n = os.path.join(root,file)
            with open(file_n,encoding='utf-8') as f:
                xml = f.read()
            xml = '<ROOT>' + xml + '</ROOT>'
            rt = ElementTree.fromstring(xml)
            for doc in rt:
                l = doc.find('TEXT').text
            j = l.split('.')
            for i in range(len(j)):
                j[i] = ' '.join(j[i].split())
                for sym in en_stop:
                    if(' '+sym+' ' in j[i]):
                        j[i] = j[i].replace(' '+sym+' ',' ')
            tot_en+=len(j)
            final_list_eng.extend(j)
final_list_eng = list(filter(lambda x: len(x)>0, final_list_eng))

In [5]:
##Hindi corpus generating as a list of lists(sentence)
tot_hn=0
final_list_hin = []
for ids in range(176,226):
    for root,dirs,files in os.walk('thesis/rel/'+str(ids)+'/hn/'):
        for file in files:
            file_n = os.path.join(root,file)
            with open(file_n,encoding='utf-8') as f:
                xml = f.read()
            xml = '<ROOT>' + xml + '</ROOT>'
            rt = ElementTree.fromstring(xml)
            for doc in rt:
                l = doc.find('TEXT').text
            j = l.split('।')
            for i in range(len(j)):
                j[i] = ' '.join(j[i].split())
                for sym in hn_stop:
                    if(' '+sym+' ' in j[i]):
                        j[i] = j[i].replace(' '+sym+' ',' ')
            tot_hn+=len(j)
            final_list_hin.extend(j)
final_list_hin = list(filter(lambda x: len(x)>0, final_list_hin))

In [6]:
##Bengali corpus generating as a list of lists(sentence)
tot_bn=0
final_list_ben = []
for ids in range(176,226):
    for root,dirs,files in os.walk('thesis/rel/'+str(ids)+'/bn/'):
        for file in files:
            file_n = os.path.join(root,file)
            with open(file_n,encoding='utf-8') as f:
                xml = f.read()
            xml = '<ROOT>' + xml + '</ROOT>'
            rt = ElementTree.fromstring(xml)
            for doc in rt:
                l = doc.find('TEXT').text
            j = l.split('।')
            for i in range(len(j)):
                j[i] = ' '.join(j[i].split())
                for sym in bn_stop:
                    if(' '+sym+' ' in j[i]):
                        j[i] = j[i].replace(' '+sym+' ',' ')
            tot_bn+=len(j)
            final_list_ben.extend(j)
final_list_ben = list(filter(lambda x: len(x)>0, final_list_ben))

In [7]:
final = final_list_eng + final_list_hin + final_list_ben

In [27]:
##These are just queries in different languages
e = []
h = []
b = []
for i in range(5):
    f = open('e_q_'+str(i),'rb')
    curr = pickle.load(f)
    e.append(curr)
    f = open('h_q_'+str(i),'rb')
    curr = pickle.load(f)
    h.append(curr)
    f = open('b_q_'+str(i),'rb')
    curr = pickle.load(f)
    b.append(curr)

In [29]:
#Language indicators to pass to laser for tokenization
langs = []
for i in range(len(final_list_eng)):
    langs.append('en')
for i in range(len(final_list_hin)):
    langs.append('hi')
for i in range(len(final_list_ben)):
    langs.append('bn')

In [11]:
#Embeddings generations and transforming them to KeyedVectors for computing similarity
laser = le.Laser()
embeddings = laser.embed_sentences(final,lang=langs)
# lang_based_keys = [sent for sent in new_sent]
# sent_vec = KeyedVectors(vector_size=embeddings.shape[1])
# sent_vec.add(lang_based_keys,embeddings)

In [14]:
Lang_based_keys = [sent for sent in final]
sent_vecs = KeyedVectors(vector_size=embeddings.shape[1])
sent_vecs.add(Lang_based_keys,embeddings)

In [368]:
query_1 = ' '.join(e[0][9])
query_2 = ' '.join(h[0][9])

In [30]:
# def check(curr_sim_words,topic_sim,tar_word,total_words_rqd):
#     heapq.heappush(curr_sim_words,(topic_sim,tar_word))
#     while(len(curr_sim_words)>total_words_rqd):
#         heapq.heappop(curr_sim_words)
#     return curr_sim_words

In [15]:
a = 'एपल आईपॉड टच एपल वाइडस्क्रीन मीडियाप्लेयर है'
b = 'The iPhone SDK, set programming tools developers, enhanced support development iPad'
# sent_vecs.similarity(a,b)
vec_a = sent_vecs[a]
vec_b = sent_vecs[b]
dot = np.dot(vec_a,vec_b)
norm_a = np.linalg.norm(sent_vecs[a])
norm_b = np.linalg.norm(sent_vecs[b])
denom = norm_a*norm_b
cos = dot/denom
cos

0.5275185124360704

In [16]:
sent_vecs.similar_by_vector(vec_b)

[('Poborsky played minutes, 291 minutes, Czech Republic Euro 2004', 1.0),
 ('Karel Poborsky, 98 caps, Czech player reach century caps Euro 2004 provided nation reaches final plays matches',
  0.7240390181541443),
 ('Eriksson delivered Beckham floppedSadly Germany Italy, players feature sought-after bunch Euro 2004',
  0.7213137149810791),
 ('Cocu, Jaap Stam, Giovanni van Bronckhorst Edwin van der Sar played minute countrys Euro 2004 matches',
  0.7212564945220947),
 ('In one-day internationals year, Yuvraj managed 96 runs, 74 coming match Sri Lanka Dhaka',
  0.7157318592071533),
 ('Desailly, turn 36 September, 222 appearances years Chelsea',
  0.7156385183334351),
 ('With goals Euro 2004, Ruud van Nistelrooy joint-second scorers list goal Czech Republics Milan Baros',
  0.7155148983001709),
 ('In Palermo, superb 89th-minute goal Tomas Rosicky gave Czech Republic 2-2 draw Italy',
  0.7111214995384216),
 ('In major currency shift introduction Cyprus pound British colonial rulers 1878, Cy

In [19]:
d = 'Poborsky played minutes, 291 minutes, Czech Republic Euro 2004'
sent_vecs.similarity(b,d)

0.45118295308025186

In [23]:
import gensim
print(gensim.__version__)

3.8.1


In [25]:
pob_sent = 'Poborsky played minutes, 291 minutes, Czech Republic Euro 2004'
iphone_sent = 'The iPhone SDK, set programming tools developers, enhanced support development iPad'
print('-')
print(len(sent_vecs.vocab))
print(sent_vecs.vocab[pob_sent].index)
print(sent_vecs.vocab[iphone_sent].index)
print('--')
print(sent_vecs.most_similar(pob_sent, topn=3))
print(sent_vecs.similar_by_vector(sent_vecs[pob_sent], topn=3))
print('---')
print(sent_vecs.most_similar(iphone_sent, topn=3))
print(sent_vecs.similar_by_vector(sent_vecs[iphone_sent], topn=3))
print('----')
print(sent_vecs.similarity(pob_sent, iphone_sent))
print('-----')

-
197659
72687
77492
--
[('" prelude unexpected', 0.7080280184745789), ('Bhagavathy Seva 6', 0.6976345777511597), ('Two unharmed', 0.6953473687171936)]
[('"It\'s undulating 5', 0.9999999403953552), ('" prelude unexpected', 0.7080280184745789), ('Bhagavathy Seva 6', 0.6976345777511597)]
---
[('Karel Poborsky, 98 caps, Czech player reach century caps Euro 2004 provided nation reaches final plays matches', 0.7240390181541443), ('Eriksson delivered Beckham floppedSadly Germany Italy, players feature sought-after bunch Euro 2004', 0.7213137149810791), ('Cocu, Jaap Stam, Giovanni van Bronckhorst Edwin van der Sar played minute countrys Euro 2004 matches', 0.7212564945220947)]
[('Poborsky played minutes, 291 minutes, Czech Republic Euro 2004', 1.0), ('Karel Poborsky, 98 caps, Czech player reach century caps Euro 2004 provided nation reaches final plays matches', 0.7240390181541443), ('Eriksson delivered Beckham floppedSadly Germany Italy, players feature sought-after bunch Euro 2004', 0.72131

In [20]:
d = 'Poborsky played minutes, 291 minutes, Czech Republic Euro 2004'
vec_d = sent_vecs[d]
sent_vecs.similar_by_vector(vec_d)

[('"It\'s undulating 5', 0.9999999403953552),
 ('" prelude unexpected', 0.7080280184745789),
 ('Bhagavathy Seva 6', 0.6976345777511597),
 ('Two unharmed', 0.6953473687171936),
 ('"The IMF provide 5', 0.6893535852432251),
 ('The decadal rise 2', 0.6862850189208984),
 ('It painstaking work', 0.6851921081542969),
 ('But Dibu unduly concerned', 0.6760345697402954),
 ('The offensive sparked exodus 2', 0.672282338142395),
 ("It's cauldron", 0.6628783941268921)]

In [21]:
sent_vecs.similar_by_vector(vec_b)

[('Poborsky played minutes, 291 minutes, Czech Republic Euro 2004', 1.0),
 ('Karel Poborsky, 98 caps, Czech player reach century caps Euro 2004 provided nation reaches final plays matches',
  0.7240390181541443),
 ('Eriksson delivered Beckham floppedSadly Germany Italy, players feature sought-after bunch Euro 2004',
  0.7213137149810791),
 ('Cocu, Jaap Stam, Giovanni van Bronckhorst Edwin van der Sar played minute countrys Euro 2004 matches',
  0.7212564945220947),
 ('In one-day internationals year, Yuvraj managed 96 runs, 74 coming match Sri Lanka Dhaka',
  0.7157318592071533),
 ('Desailly, turn 36 September, 222 appearances years Chelsea',
  0.7156385183334351),
 ('With goals Euro 2004, Ruud van Nistelrooy joint-second scorers list goal Czech Republics Milan Baros',
  0.7155148983001709),
 ('In Palermo, superb 89th-minute goal Tomas Rosicky gave Czech Republic 2-2 draw Italy',
  0.7111214995384216),
 ('In major currency shift introduction Cyprus pound British colonial rulers 1878, Cy

In [516]:
# vv = sent_vecs['एपल आईपॉड टच एपल वाइडस्क्रीन मीडियाप्लेयर है']
# print(vv)
# print(n_e[0])
# cos = np.dot(vv,n_e[0])/(np.linalg.norm(vv)*np.linalg.norm(n_e[0]))
# cos

In [532]:
eng_sent_dict,hin_sent_dict,ben_sent_dict = {},{},{}
for sent in final_list_eng:
    eng_sent_dict[sent]=1
for sent in final_list_hin:
    hin_sent_dict[sent]=1
for sent in final_list_ben:
    ben_sent_dict[sent]=1

In [31]:
# def create_queries(fold,queries,src_lang,tar_dict):
#     f = open('fold_test_set_2012','rb')
#     test_set = pickle.load(f)
#     test_qry = test_set[fold]
#     f.close()   
#     j=0
#     st = '<topics>\n'
#     for sent in queries:
#         qry = ' '.join(sent)
#         new_e = laser.embed_sentences([qry],lang=src_lang)
#         ret = sent_vecs.similar_by_vector(new_e[0],topn=5000)
#         similar_sent = find_similar(ret,tar_dict)
# #         print(similar_sent)
#         tp = '<top>\n'
#         nm = '<num>' + str(test_qry[j]) + '</num>\n'
#         try:
#             ttle = '<title>' + similar_sent + '</title>\n'
#         except TypeError:
#             print('nothing found for ',qry)
#         tp_end = '</top>\n'
#         st+=(tp+nm+ttle+tp_end)
#         j+=1
#     st+='</topics>'
#     return st

In [32]:
# def save(s,folds,src,tar):
#     f = open('Desktop/Laser/topic/'+str(folds)+'_'+src+'_'+tar,'w')
#     f.write(s)
#     f.close()

In [33]:
# for folds in range(5):
#     s = create_queries(folds,e[folds],'en',hin_sent_dict)
#     save(s,folds,'en','hi')
#     s = create_queries(folds,e[folds],'en',ben_sent_dict)
#     save(s,folds,'en','bn')
#     s = create_queries(folds,h[folds],'hi',eng_sent_dict)
#     save(s,folds,'hi','en')
#     s = create_queries(folds,h[folds],'hi',ben_sent_dict)
#     save(s,folds,'hi','bn')
#     s = create_queries(folds,b[folds],'bn',eng_sent_dict)
#     save(s,folds,'bn','en')
#     s = create_queries(folds,b[folds],'bn',hin_sent_dict)
#     save(s,folds,'bn','hi')

In [40]:
# terrier_en = 'terrier-core_en/'
# terrier_hn = 'terrier-core_hn/'
# terrier_bn = 'terrier-core_bn/'

In [39]:
# def save_ret(fold,src,tar,terrier_loc):
#     os.system('cp '+terrier_loc+'var/results/DPH_0.res Desktop/Laser/ret/'+str(fold)+'_'+src+'_'+tar)
#     os.system('rm -r '+terrier_loc+'var/results/*')

In [38]:
# def create_ret(fold):
#     for lang in ['en','hi','bn']:
#         if(lang=='en'):
#             src_1 = 'hi'
#             src_2 = 'bn'
#             terrier_loc = terrier_en
#         elif(lang=='hi'):
#             src_1 = 'en'
#             src_2 = 'bn'
#             terrier_loc = terrier_hn
#         else:
#             src_1 = 'en'
#             src_2 = 'hi'
#             terrier_loc = terrier_bn
#         f_1 = 'Desktop/Laser/topic/'+str(fold)+'_'+src_1+'_'+lang
#         os.system(terrier_loc+'bin/terrier batchretrieval -t '+f_1)
#         save_ret(fold,src_1,lang,terrier_loc)
#         print(fold,src_1,lang,' done ')
#         f_2 = 'Desktop/Laser/topic/'+str(fold)+'_'+src_2+'_'+lang
#         os.system(terrier_loc+'bin/terrier batchretrieval -t '+f_2)
#         save_ret(fold,src_2,lang,terrier_loc)
#         print(fold,src_2,lang,' done ')

In [37]:
# def remove_dupli(file_name,fold,src,tar):    
#     d = defaultdict(list)
#     f = open(file_name)
#     for line in f:
#         lst = line.split()
#         d[lst[0]].append([lst[1],lst[2],lst[4],lst[5]])
#     st = ''
#     for x in d:
#         temp = {}
#         i=0
#         for lst in d[str(x)]:
#             if(lst[1] in temp):
#                 continue
#             temp[lst[1]]=1
#             t = str(x)+' '+lst[0]+' '+lst[1]+' '+str(i)+' '+lst[2]+' '+lst[3]+'\n'
#             st+=t
#             i+=1
#     f = open('Desktop/Laser/dupli_removed/'+str(fold)+'_'+src+'_'+tar,'w')
#     f.write(st)

In [36]:
# def create_map(fold):
#     for lang in ['en','hi','bn']:
#         if(lang=='en'):
#             src_1 = 'hi'
#             src_2 = 'bn'
#             terrier_loc = terrier_en
#             rel_file = 'thesis/qrels/en.qrels.176-225.2012-v2.txt'
#         elif(lang=='hi'):
#             src_1 = 'en'
#             src_2 = 'bn'
#             terrier_loc = terrier_hn
#             rel_file = 'thesis/qrels/hi.qrels.176-225.2012-v2.txt'
#         else:
#             src_1 = 'en'
#             src_2 = 'hi'
#             terrier_loc = terrier_bn
#             rel_file = 'thesis/qrels/bn.qrels.176-225.2012-v2.txt'
#         f_1_old = 'Desktop/Laser/ret/'+str(fold)+'_'+src_1+'_'+lang
#         remove_dupli(f_1_old,fold,src_1,lang)
#         f_1_new = 'Desktop/Laser/dupli_removed/'+str(fold)+'_'+src_1+'_'+lang
#         os.system('trec_eval/trec_eval -q '+rel_file+' '+f_1_new+' > Desktop/Laser/map/'+str(fold)+'_'+src_1+'_'+lang)
#         f_2_old = 'Desktop/Laser/ret/'+str(fold)+'_'+src_2+'_'+lang
#         remove_dupli(f_2_old,fold,src_2,lang)
#         f_2_new = 'Desktop/Laser/dupli_removed/'+str(fold)+'_'+src_2+'_'+lang
#         os.system('trec_eval/trec_eval -q '+rel_file+' '+f_2_new+' > Desktop/Laser/map/'+str(fold)+'_'+src_2+'_'+lang)

In [35]:
# for folds in range(5):
#     create_map(folds)

In [34]:
# st = ''
# fold=4
# config=''
# st+=',,'+config+'\n'
# print('\t\t\tMAP\tBpref')
# st+=',MAP,Bpref\n'
# for lang in ['en','hi','bn']:
#     if(lang=='en'):
#         tar_1 = 'hi'
#         tar_2 = 'bn'
#     elif(lang=='hi'):
#         tar_1 = 'en'
#         tar_2 = 'bn'
#     else:
#         tar_1 = 'en'
#         tar_2 = 'hi'
#     file_1 = 'Desktop/Laser/map/'+str(fold)+'_'+lang+'_'+tar_1
#     f = open(file_1)
#     for line in f:
#         spl = line.split()
#         if(spl[1]=='all' and spl[0]=='map'):
#             c_map_1 = spl[2]
#         if(spl[1]=='all' and spl[0]=='bpref'):
#             c_bpref_1 = spl[2]
#     f.close()
#     print(lang+'->'+tar_1,'\t',c_map_1,'\t',c_bpref_1)
#     st+=lang+'->'+tar_1+','+c_map_1+','+c_bpref_1+'\n'
#     file_2 = 'Desktop/Laser/map/'+str(fold)+'_'+lang+'_'+tar_2
#     f = open(file_2)
#     for line in f:
#         spl = line.split()
#         if(spl[1]=='all' and spl[0]=='map'):
#             c_map_2 = spl[2]
#         if(spl[1]=='all' and spl[0]=='bpref'):
#             c_bpref_2 = spl[2]
#     f.close()
#     print(lang+'->'+tar_2,'\t',c_map_2,'\t',c_bpref_2)
#     st+=lang+'->'+tar_2+','+c_map_2+','+c_bpref_2+'\n'